In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import pandas as pd
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from gensim.models import KeyedVectors

Using TensorFlow backend.
C:\Users\Dime\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

In [4]:
print('Indexing word vectors.')

Indexing word vectors.


In [5]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [6]:
(embeddings_index['refrigerator'])

array([-0.89757 ,  0.1165  ,  0.10973 , -0.021751, -0.24992 ,  0.24924 ,
        0.88088 ,  0.55585 ,  0.70756 ,  0.10431 ,  0.295   , -0.2227  ,
        0.10154 ,  1.0233  ,  0.075921, -0.039672,  0.086631,  0.33136 ,
       -0.035846, -0.88137 , -0.23373 ,  0.5915  , -0.21436 , -0.32878 ,
        0.59057 ,  0.18832 , -0.57551 , -0.27209 , -0.52268 , -0.060988,
        0.45458 , -0.060737,  0.10032 ,  0.43078 , -0.73463 ,  0.89155 ,
        0.64138 , -0.64439 ,  0.60504 , -1.1     ,  0.97867 , -0.7893  ,
       -0.18588 , -0.33417 ,  0.093798,  0.23285 , -0.51151 , -0.024952,
        0.43589 , -0.36042 ,  0.31272 , -0.41539 , -0.36774 ,  0.75    ,
       -0.46198 , -0.53486 , -0.34307 , -0.078263,  0.83441 ,  0.45955 ,
        0.40042 ,  0.25485 , -0.18669 ,  0.94086 ,  0.82562 ,  0.57476 ,
        0.74191 ,  0.11995 , -0.77276 ,  0.1418  ,  0.1188  , -0.36804 ,
        1.4014  ,  0.48464 , -0.080731,  1.1573  ,  0.46906 , -1.2566  ,
        0.6941  ,  0.20686 ,  0.25455 , -0.076509, 

In [7]:
data = pd.read_csv("C:\\Users\\Dime\\Desktop\\Insight project\\insight\\data\\temp_data.csv",  encoding="utf8")

In [8]:
text = data['label']
cls = data['type']
porter = PorterStemmer()
lancaster=LancasterStemmer()

In [9]:
#lancaster.stem()
for i in range(len(text)):
    temp = text[i]
    temp = temp.split(" ")
    preprocessed = ''
    for word in temp:
        word = word.lower()
        #word = porter.stem(word)
        
        if(len(preprocessed) == 0):
            preprocessed = word 
        else:
            preprocessed = preprocessed +" "+ word
        
    
    text[i] = preprocessed
        
        



C:\Users\Dime\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [10]:
embed = np.zeros((len(text), 500))

In [11]:
for i in range(len(text)):
    temp = text[i]
    temp = temp.split(" ")
    #print(temp)
    for j in range(len(temp)):
        x = temp[j]
        if(len(x) > 0):
            print(x)
            p = embeddings_index[x]
            print(p)
            
            embed[i][(j*100):((j+1)*100)] = p

air
[-1.0209e+00 -2.2182e-01  4.5351e-01 -1.1121e-01  3.0888e-02 -3.1316e-01
  7.8091e-02  1.2609e-01 -7.9106e-01  4.8882e-01  5.1345e-01 -1.0514e+00
 -1.4050e-01  5.0340e-01  5.1471e-02 -9.3455e-01  4.5541e-01 -5.1351e-01
 -4.3171e-02 -3.8542e-01  4.8001e-01  2.5086e-03  4.9763e-01 -1.9554e-01
  1.4958e-01  1.5754e-01 -2.4000e-01  2.7557e-01  7.8082e-01 -2.7568e-01
 -2.8353e-01  7.2381e-01 -3.4221e-01  1.6688e-01 -2.3963e-01  5.7995e-01
  5.0133e-01  2.4512e-01  4.0543e-01  1.1763e-01 -5.0907e-01 -3.2694e-01
 -4.7457e-01  5.2046e-01  1.1700e+00  9.3120e-01 -1.7163e-01 -2.3183e-01
  1.4042e-01 -8.6031e-01  1.7651e-01  4.8482e-01 -1.7385e-01  1.2666e+00
 -4.1413e-03 -2.0092e+00  8.6051e-01  1.8135e-01  2.8750e+00  4.8253e-01
  3.6204e-01  9.3870e-01  4.0597e-01  6.0482e-01  1.1505e-01 -5.7461e-02
 -5.7051e-02 -8.1308e-01 -4.2144e-01  8.4546e-01  4.0318e-01 -1.6134e+00
  5.0362e-01 -3.6898e-01  3.8404e-01  3.8157e-01  7.8736e-01 -9.3565e-02
 -9.1532e-01  5.6967e-01  5.0934e-01 -6.4488e-0

  4.3171e-01 -7.4729e-01  1.5500e+00 -6.3256e-01]
phone
[ 3.1764e-02 -6.0768e-01  5.2233e-01 -1.1533e-02  3.6009e-01  3.6460e-01
 -4.9728e-03 -3.3769e-04  6.6011e-01 -1.2602e-01  2.3832e-01  5.6113e-02
 -1.1328e-01  3.5199e-01  2.4070e-01 -2.9588e-01 -3.1811e-01  7.9509e-03
  3.2996e-01 -1.0383e-01 -4.0230e-01 -3.7351e-03  4.7088e-01  2.2141e-01
  3.3043e-01 -4.5048e-01  3.5376e-01  5.5943e-01  2.3509e-01  5.0190e-02
  5.7384e-01  9.1137e-01  8.9360e-01  1.3000e-01  6.7807e-01  4.1787e-01
 -6.9812e-01 -6.0581e-01  1.1147e+00 -4.3455e-03  4.6439e-01 -3.8663e-01
  9.2078e-02 -3.5278e-01 -9.2302e-01  3.7423e-02 -4.9481e-01 -2.0403e-01
  8.0609e-01 -6.7063e-01  1.9324e-01  6.9329e-01  8.1743e-01  3.7762e-01
  2.6951e-01 -1.7669e+00 -7.0825e-01  2.7024e-01  1.9455e+00  7.2376e-01
  1.7558e-01 -1.7475e-01  1.7004e-01 -6.7982e-01 -2.3057e-01  8.5733e-02
  7.8184e-01  3.4410e-01  8.3690e-01  2.7714e-01 -4.1911e-01 -1.3996e-02
  3.6299e-01 -1.2430e+00  4.0935e-01  8.2805e-01  6.8355e-01 -2.0146

sheets
[-1.0347     0.55974   -0.4257    -0.42311   -0.20353    0.46689
  0.054739  -0.23409   -0.9743     0.16455    1.0203     0.23161
  0.091184   0.96161    0.21432    0.0090782  0.50702   -0.089223
  0.49702   -1.2858    -0.27204    0.8213     0.25071    0.17628
  0.11315   -0.30406   -0.23973   -0.24548   -0.72832   -0.41897
  1.2141     0.16523   -0.16717   -0.034763  -0.65518   -0.25335
  0.58778   -0.056613  -0.30468   -0.693      0.15259   -0.68016
 -0.77939    0.39387   -0.54061   -0.39044   -0.57554   -0.17086
  0.11374   -0.30857   -0.22966    0.28426   -0.025899   0.83877
 -0.12276   -0.84991    0.12688   -0.23466    0.87539    0.7726
 -0.73056    0.69828   -0.72491    0.66066    0.222      0.12067
  0.66692    0.49139    0.06416   -1.4125    -0.26483    0.38812
  0.27298    0.18163    0.93629    1.2075     0.15283    0.32952
  0.37856    0.39795    0.4199     0.55408   -0.93837   -0.043726
 -0.31398   -0.4607     0.39777   -0.26228    0.15863    0.532
  0.10803    0.0502

 -0.48349   -0.024972   0.70715    0.80798  ]
hangers
[ 0.068252   0.33365   -0.13592   -1.0584     0.015549   0.476
 -0.2225     0.2065     0.13652    0.44586    0.52682   -0.048287
  0.41465    0.71778    0.81579   -0.03217    0.43349   -0.2462
  0.66675   -0.96817   -0.31188    0.51488   -0.21418   -0.66917
  0.4826    -0.044407  -0.45633   -0.12428   -0.23073   -0.64254
  0.25734   -0.71848   -0.054609   0.49692    0.38684    0.43499
  0.17487   -0.13091    0.41752   -0.44865    0.13928   -0.43953
 -0.4133     0.0015419 -0.91619   -0.35692   -0.46179    0.18314
  0.38923    0.31948   -0.41644    0.030578  -0.24232    0.5353
 -0.27953    0.85496    0.84933   -0.39631    0.25499    0.0057584
 -0.23589    0.68931   -0.34792    1.1419    -0.015475  -0.19012
  0.49332    0.55129   -0.35445   -0.50643   -0.44055   -0.45527
  0.27914   -0.50783   -0.11089   -0.29477    0.36242   -0.15797
  0.26812    0.17733   -0.028936  -0.050175   0.24399    0.097048
 -0.16733   -0.39984   -0.61399    0

  0.076995   0.21369    0.24099    0.003914 ]
plant
[-0.26587   0.76435   0.38179  -0.17323   0.013872 -1.237     0.36181
  0.80811   0.094433  0.63863  -0.52189   0.14387   0.41282   0.13564
 -0.48617  -0.20985   0.82291   0.25933   0.34834   0.24553  -0.13497
 -0.12786   0.17731   0.69626   0.17722   0.17209  -0.040392  0.24483
  0.042582  0.90519  -1.1165    0.46325   0.57555  -0.21269  -0.46593
  0.49176   0.38885  -1.096     0.23474  -0.060113  0.25634  -1.2045
 -0.28458  -0.59228   0.58706   0.47478   0.40608   0.16202  -0.43319
 -0.18207   0.16367  -0.78588  -0.22281   0.78211  -0.27724  -0.75947
 -1.2059   -0.5484    1.7789   -0.21905   0.86618   0.72736   0.13096
  0.3253    1.421     0.060767  0.47491  -0.22603   0.048039 -0.25337
 -0.1242    0.23913  -0.75115  -0.43663   0.51712   0.26971   1.0175
 -0.49526  -0.88226   1.4675    0.53784  -0.34357  -1.2434    0.64174
 -0.85184   0.59572   1.2472   -0.21262  -0.024009  0.59812   0.51751
 -0.92531   0.017226  0.55508  -0.64655 

...

food
[-1.6486e-01  9.1997e-01  2.2737e-01 -4.9031e-01 -1.8082e-03 -3.3803e-01
  5.7221e-02  1.4601e-01  4.0202e-01 -2.8858e-01 -4.7495e-01 -5.6369e-01
  2.7037e-01  5.1702e-01 -1.1241e-01  1.8314e-01  2.2066e-01 -4.8606e-01
 -8.7284e-01 -6.2587e-02  4.3016e-02  2.3641e-01  5.9705e-01 -3.8640e-01
 -2.5194e-01  9.6862e-01 -4.3112e-01 -4.8370e-01 -1.1396e+00  9.2425e-02
 -1.1476e-01 -7.4291e-02 -6.2524e-02 -9.5122e-02 -2.2714e-01  8.8291e-01
  3.9978e-01  7.6631e-01 -6.7697e-01 -6.2829e-01 -1.1872e-01 -2.4492e-01
 -5.8893e-01 -8.5088e-01  1.1107e+00  4.2190e-01 -1.5072e+00 -1.9509e-01
 -2.6712e-01 -7.0801e-01  5.5075e-01 -4.6929e-02 -2.5203e-01  7.4411e-01
 -1.8325e-01 -1.4885e+00 -4.6393e-01 -1.0338e-01  2.3525e+00 -1.5421e-01
  3.9833e-01  1.5344e-02  8.0708e-02 -2.7373e-01  9.7057e-01 -1.9383e-02
  2.0899e-01 -6.4033e-01  9.2509e-01 -4.5371e-01 -7.0564e-01 -1.6033e-01
 -7.1761e-02  6.2856e-01  3.5732e-01  8.8802e-01 -6.9127e-01  4.9634e-02
 -9.3347e-01  6.5396e-01  3.7165e-01  5.8363e-

  0.86807   0.13589 ]
filters
[-0.45419   -0.082873  -0.62792   -0.80638    0.21177    0.33859
 -0.53082    0.093712   0.37407    0.063257   0.34349   -1.1291
 -0.66237   -0.1743     0.34123   -0.1927     0.40571    0.098099
  0.49123   -0.61133   -0.67699   -0.51802    0.65887    0.40772
  0.4072    -0.15842    0.48243    0.4416    -0.67775   -0.78312
  1.0075     0.51166    0.015832   0.1352     0.58794    0.48605
  0.18959   -0.39703   -0.022209   0.0036925 -0.29838    0.046529
 -1.4081    -0.49343   -0.36992    0.32124   -0.17191    0.071049
  0.27579   -0.46141    0.43157    0.21901   -0.44895    0.89777
 -0.32877   -0.076841   0.44363    0.25512    0.82993   -0.24258
  0.18869    0.19627    0.42418    0.88982    0.63573    0.3521
  0.43475   -0.94365   -0.20359   -0.63703    0.097199  -0.27225
  1.2739     0.28459    0.16602   -0.12269    0.44334   -0.14882
  0.30824    0.32996    0.61616   -0.82713   -1.0016     0.13481
 -0.8064     0.96632    1.075     -0.41888   -0.49201   -0.

[-0.44645    1.1511    -0.096384  -0.62197    0.25974    0.88332
 -0.14543   -0.0093618  0.24674    0.53501   -0.082318  -0.23291
  0.075901   0.4264     0.51668    0.449      0.95047    0.36059
  1.2024    -1.2026     0.52799    0.66658   -0.20577   -0.29893
  0.73577   -0.4353     0.040796   0.17296   -0.85485    0.062584
  0.84478   -0.099981   0.26971   -0.16943    0.029988   0.35439
  0.67056   -0.054509   0.27704   -0.66393    0.13564   -0.50349
  0.1087    -0.15851   -0.28774   -0.19482   -0.91591    0.36382
 -0.79831   -0.0435    -0.018412  -0.10187    0.43086    0.86657
  0.0079583 -0.46867   -0.41695    0.059395   0.2073    -0.17013
 -0.45625    0.75277   -0.53766    0.5905    -0.17651   -0.38577
  0.46239    0.6331    -0.76745   -0.76176   -0.15033    0.44099
  0.75917    0.048803   0.11941    1.2074     0.64172    0.65831
  0.34778    0.63034    0.13712    0.14093   -0.077197   0.46031
 -0.63973   -0.84766    0.48205   -0.28974   -0.40393    0.34579
  0.47668    0.018281   

 -0.62982   -0.50962    0.77714   -0.54894  ]
lumber
[-0.30274    0.2465     0.04248   -0.24256    0.028116  -1.7044
 -0.19321    0.030052  -0.25695   -0.60521    0.17347   -0.21397
  0.24614    0.056493  -0.15178    0.099608   0.35073   -0.1465
 -0.32381   -0.42937    0.72433    0.8204     1.3958    -0.16407
  0.46688   -0.23717   -0.61346   -0.43439   -0.35891    0.065167
 -0.24313    0.69967    0.21854   -0.858     -0.51434    0.66224
 -0.12405   -0.56629    0.87347   -0.42163    0.074015  -0.41132
 -0.23134   -0.25265    0.12688    0.17295   -0.37673   -0.23972
  0.077469  -0.11028   -0.31044   -0.48571    0.11073    0.14687
  0.04984    0.59066   -0.81595   -0.73005    1.5342    -0.11689
  0.063557  -0.063793  -0.51602    0.45837   -0.083371  -0.76278
  0.84431    0.02659    0.18725   -0.67871   -0.59358    0.4868
  0.829      0.68687    0.28004   -0.42503    0.41986    0.8006
 -0.91096    1.0633     0.024061   0.88056   -0.050464   0.96759
  0.45702   -0.50862    0.079315  -0.721

  0.36791   0.97297 ]
